<a href="https://colab.research.google.com/github/Ramandeep-Singh17/DLusingPyTorch/blob/main/5_pytorch_training_pipeline_using_dataset_and_dataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔥 Recap + Why Dataset & DataLoader Needed

---

# 🔹 🔁 Recap (Flow till now)

1. Tensors → Data ko numerical form me store kiya  
2. Autograd → Automatic gradients (derivatives)  
3. nn Module → Neural network easily build kiya  
4. Training Pipeline → forward → loss → backward → update  

---

# 🔹 Abhi hum kya kar rahe the?

Hum **Batch Gradient Descent** use kar rahe the:

👉 Pure dataset ek saath use karke update

---

## ❌ Problem with Batch Gradient Descent

1. Memory inefficient  
   → Large dataset RAM/GPU me fit nahi hota  

2. Slow convergence  
   → Har update ke liye poora data use ho raha hai (slow learning)

---

# 🔹 Solution → Mini-Batch Gradient Descent

👉 Data ko chhote batches me tod dete hain

Example:
1000 rows → batch size = 100  
→ 10 batches  

Ab:
- Har batch ke baad update  
- Faster + efficient  

---

# 🔹 Tumne kya kiya (Manual Mini-Batch Code)

Tumne manually:

- start_idx, end_idx use kiya  
- slicing se batch banaya  
- loop chalaya  

✔ Concept correct hai  
❌ But scalable nahi hai  

---

# 🔴 Problems with Manual Approach

## 1. No Standard Interface for Data

Har baar:
- alag tareeke se data load karna padta hai  
- koi fixed structure nahi  

👉 Messy code

---

## 2. No Easy Way to Apply Transformations

Real-world me:

- image resize karna
- normalization
- augmentation

Manual code me:
❌ Hard to manage

---

## 3. Shuffling & Sampling Problem

Training me:

- data shuffle hona chahiye
- random sampling hona chahiye

Manual me:
❌ khud likhna padega (error-prone)

---

## 4. Batch Management & Parallelization

- batch banana manually
- multi-core loading nahi
- slow data pipeline

👉 Large scale DL me fail ho jayega

---

# ❌ Final Problem

👉 Ye approach scalable nahi hai  
👉 Real-world deep learning me use nahi hota  

---

# 🔥 Solution → Dataset + DataLoader

In sab problems ko solve karne ke liye:

👉 PyTorch deta hai:

- Dataset → data ko structure deta hai  
- DataLoader → batching, shuffle, fast loading  

---

# 🔥 Hinglish Final Understanding

Batch gradient descent slow aur memory heavy tha  
isliye hum mini-batch use kar rahe hain  

Par manual batching scalable nahi hai  
isliye PyTorch me Dataset aur DataLoader use karte hain  
jo data handling ko easy, fast aur professional bana deta hai


In [ ]:
import numpy as np
import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/gscdit/Breast-Cancer-Detection/refs/heads/master/data.csv')
df.head()

,id,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,...,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst,Unnamed: 32
0,842302,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,NaN
1,842517,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,NaN
2,84300903,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,NaN
3,84348301,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,NaN
4,84358402,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,NaN


In [ ]:
df.shape

(569, 33)

In [ ]:
df.drop(columns=['id', 'Unnamed: 32'], inplace= True)

In [ ]:
df.head()

,diagnosis,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave points_mean,symmetry_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave points_worst,symmetry_worst,fractal_dimension_worst
0,M,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,M,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,M,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,M,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,M,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


### train test split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df.iloc[:, 1:], df.iloc[:, 0], test_size=0.2)

### scaling

In [ ]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [ ]:
X_train

array([[ 2.52815489,  0.64264222,  2.41247655, ...,  1.82459706,
         1.12089035,  0.44606887],
       [ 0.5431291 , -1.04056526,  0.48135452, ..., -0.0286167 ,
        -0.19683055, -0.74259412],
       [ 1.65396401,  2.09845258,  1.56811566, ...,  0.73124662,
        -0.53154141, -0.94753602],
       ...,
       [ 0.21461151, -0.04658202,  0.18764621, ...,  0.51958129,
         0.10213448,  0.05531299],
       [-0.9017915 ,  0.48088551, -0.82760686, ...,  0.42060109,
         0.1053841 ,  0.67505729],
       [-1.33192682,  0.56293601, -1.32452738, ..., -1.17374139,
        -0.04409841, -0.03650098]])

In [ ]:
y_train

,diagnosis
164,M
406,B
565,M
507,B
516,M
...,...
107,B
201,M
291,B
41,M


### Label Encoding

In [ ]:
encoder = LabelEncoder()
y_train = encoder.fit_transform(y_train)
y_test = encoder.transform(y_test)

In [ ]:
y_train

array([1, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1,

### Numpy arrays to PyTorch tensors

In [ ]:
X_train_tensor = torch.from_numpy(X_train.astype(np.float32))
X_test_tensor = torch.from_numpy(X_test.astype(np.float32))
y_train_tensor = torch.from_numpy(y_train.astype(np.float32))
y_test_tensor = torch.from_numpy(y_test.astype(np.float32))

In [ ]:
X_train_tensor.shape

torch.Size([455, 30])

In [ ]:
y_train_tensor.shape

torch.Size([455])

In [ ]:
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):

  def __init__(self, features, labels):

    self.features = features
    self.labels = labels

  def __len__(self):

    return len(self.features)

  def __getitem__(self, idx):

    return self.features[idx], self.labels[idx]



In [ ]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

In [ ]:
train_dataset[10]

(tensor([ 1.4925,  0.9169,  1.4752,  1.4122,  0.3250,  0.5202,  1.1660,  1.3300,
          0.6221, -0.5320,  0.6756,  2.9047,  1.1882,  0.5850,  1.3589,  0.8473,
          1.3375,  3.5864,  0.8360,  0.8585,  0.9084,  0.7877,  1.0060,  0.7160,
         -0.2976, -0.0678,  0.5276,  1.0312, -0.5250, -0.4202]),
 tensor(1.))

In [ ]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=True)

### Defining the model

In [ ]:
import torch.nn as nn


class MySimpleNN(nn.Module):

  def __init__(self, num_features):

    super().__init__()
    self.linear = nn.Linear(num_features, 1)
    self.sigmoid = nn.Sigmoid()

  def forward(self, features):

    out = self.linear(features)
    out = self.sigmoid(out)

    return out

### Important Parameters

In [ ]:
learning_rate = 0.1
epochs = 25

In [ ]:
# create model
model = MySimpleNN(X_train_tensor.shape[1])

# define optimizer
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

# define loss function
loss_function = nn.BCELoss()

### Training Pipeline

In [ ]:

# define loop
for epoch in range(epochs):

  for batch_features, batch_labels in train_loader:

    # forward pass
    y_pred = model(batch_features)

    # loss calculate
    loss = loss_function(y_pred, batch_labels.view(-1,1))

    # clear gradients
    optimizer.zero_grad()

    # backward pass
    loss.backward()

    # parameters update
    optimizer.step()

  # print loss in each epoch
  print(f'Epoch: {epoch + 1}, Loss: {loss.item()}')

Epoch: 1, Loss: 0.19787999987602234
Epoch: 2, Loss: 0.10052193701267242
Epoch: 3, Loss: 0.09019561111927032
Epoch: 4, Loss: 0.04922916740179062
Epoch: 5, Loss: 0.0791965052485466
Epoch: 6, Loss: 0.17395448684692383
Epoch: 7, Loss: 0.2700732350349426
Epoch: 8, Loss: 0.09979669004678726
Epoch: 9, Loss: 0.10649565607309341
Epoch: 10, Loss: 0.08558855205774307
Epoch: 11, Loss: 0.05072442442178726
Epoch: 12, Loss: 0.015160821378231049
Epoch: 13, Loss: 0.11817531287670135
Epoch: 14, Loss: 0.025228416547179222
Epoch: 15, Loss: 0.10527627915143967
Epoch: 16, Loss: 0.3358323574066162
Epoch: 17, Loss: 0.00839468277990818
Epoch: 18, Loss: 0.01976577565073967
Epoch: 19, Loss: 0.002903284505009651
Epoch: 20, Loss: 0.005619850009679794
Epoch: 21, Loss: 0.033411893993616104
Epoch: 22, Loss: 0.051596514880657196
Epoch: 23, Loss: 0.21043191850185394
Epoch: 24, Loss: 0.07372516393661499
Epoch: 25, Loss: 0.03629717603325844


### Evaluation

In [ ]:
# Model evaluation using test_loader
model.eval()  # Set the model to evaluation mode
accuracy_list = []

with torch.no_grad():
    for batch_features, batch_labels in test_loader:
        # Forward pass
        y_pred = model(batch_features)
        y_pred = (y_pred > 0.8).float()  # Convert probabilities to binary predictions

        # Calculate accuracy for the current batch
        batch_accuracy = (y_pred.view(-1) == batch_labels).float().mean().item()
        accuracy_list.append(batch_accuracy)

# Calculate overall accuracy
overall_accuracy = sum(accuracy_list) / len(accuracy_list)
print(f'Accuracy: {overall_accuracy:.4f}')


Accuracy: 0.9705
